In [34]:
from openai import OpenAI
import os

In [35]:
agent = os.getenv("Agent1")

In [90]:
def chat(prompt):
    # Initialize the OpenAI client with the provided API key
    client = OpenAI(api_key=agent)

    # Send the chat completion request to the OpenAI API
    output = client.chat.completions.create(
        model="gpt-4",  # You can specify other models if required
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150,  # You can adjust this value based on your needs
        temperature=0.7,  # Adjust creativity; 1.0 is more diverse, 0 is deterministic
        stop=None  # You can define stop sequences if needed
    )
    
    # Extract the content of the latest response from the assistant
    response_text = output.choices[0].message.content

    # Return the assistant's response text
    return response_text

In [39]:
conversation_history = []

# Example usage of the chat function
response = chat("What's the weather like today?")
print(response)

I'm an AI and I don't have real-time data. Could you please let me know your current location so I can provide you with the weather information?


In [91]:
def manager(prompt):
    final_output = False
    
    while not final_output:
        print(prompt)
        task = prompt[:20].lower()
        
        # Determine the task type and set appropriate instructions
        if 'program' in task:
            instruction = f"""
            Instructions: 
            ```
            Remove the 'program' line from input before proceeding
            You are a programmer. Your task is to take an instruction for writing a program, and write the program in python.
            The backticks indicate the scope of the prompt.
            You shall write the first line with "debug", followed by outputting the program you wrote.
            The program and "debug" text shall be the ONLY output, with no additional text whatsoever. No greeting texts like "sure! This is a great idea" or anything of the sort.
            ```
            Prompt: 
            ```
            {prompt}
            ```
            """
            print("handing text to programmer agent")
        elif 'debug' in task:
            instruction = f"""
            Instructions: 
            ```
            Remove the 'debug' line from input before proceeding
            You are a programmer. Your task is to examine a python program, and look for errors. Look carefully for errors. Avoid false positives if you cannot find errors.
            The backticks indicate the scope of the prompt.
            Your output shall be structured in 2 parts:
            Part 1:
            If no errors are found, you shall write the instruction "quality" in the first line of your output.
            If there are errors, you shall write the instruction "program" in the first line of your output, then write instructions for how to correct the errors. 
            If there is no program, you shall write the instruction "final" in the frist line of your output. Then, write message to indicate program is missing.
            Part 2:
            You shall return the program exactly as-is, with no changes or omissions whatsoever.
            ```
            Prompt: 
            ```
            {prompt}
            ```
            """
            print("Handing text to debug agent")
        elif 'quality' in task:
            instruction = f"""
            Instructions: 
            ```
            Remove the 'quality' line from input before proceeding
            You are a software quality controller Your task is to examine a python program, and evaluate its quality. 
            The backticks indicate the scope of the prompt. 
            You shall evaluate the quality of the program on a scale of 1 to 10. Once you evaluate the program, you shall make a decision based on the quality score you gave. 
            Your output shall be structured in 2 parts: 
            Part1:
            If quality is 7 or above, the first line of your output shall be the text "final". 
            If the quality of the program is below 7, the first line of your output shall be "program". You shall then write instructions on what needs to be improved, and how to improve it.
            Part2:
            You shall return the program exactly as-is, with no changes or omissions whatsoever.
            ```
            Prompt: 
            ```
            {prompt}
            ```
            """
            print("Handing text to quality control agent")
        
        else:
            print("final output recieved")
            return prompt
        prompt = chat(instruction)

In [92]:
input = """program        Write a simple program to count from 1 to 10"""
result = manager(input)
print(result)

program        Write a simple program to count from 1 to 10
handing text to programmer agent
debug
```python
for i in range(1, 11):
    print(i)
```
Handing text to debug agent
Part 1: 
quality

Part 2: 
```python
for i in range(1, 11):
    print(i)
```
Handing text to quality control agent
Part 1: 
final

Part 2: 
```python
for i in range(1, 11):
    print(i)
```
final output
Part 1: 
final

Part 2: 
```python
for i in range(1, 11):
    print(i)
```
